# Investigation of geophysical sensor data to inform priors

Since we don't have a really great idea of what constitutes a good set of priors for real data, here I try my best to sort out what is going on using what I hope will be simple, but robust, assumptions.

In [ ]:
import GPy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
%matplotlib inline

## Noise and length scale characteristics for gravity and magnetism

We've been running with some set of priors for gravity and magnetism, but in all fairness we have no idea what those should be.  We know they're both linear sensors that integrate over rock properties, with a 3-D sensitivity profile that gets broader with depth.  So by fitting a GP to them, we get some idea of the noise, and a lower limit on the relevant length scale.  Since they're on a grid, we could also consider the autocorrelation.

This isn't really meant to be a Bayesian analysis, but it's meant to give us some idea of the order of magnitude of the noise in a model that's flexible enough to respond to changes, but that insists on smoothness so we can pick off the delta-function component of the covariance.

In [ ]:
dir_data = '/Users/davidkohn/dev/obsidian/data'
fname_grav = 'gravity_400m_Gascoyne.txt'
fname_mag = 'mag_TMI_gascoyne.txt'
lt_val_grav = 0.05
lt_val_mag = 0.015
list_fnames = [
    fname_grav,
    fname_mag
]
list_less_than_values = [
    0.05,
    0.015
]

In [ ]:
msg_str0 = 'fpath: {}'
msg_str1 = '  Latitude min: {}\n  Latitude max: {}'
msg_str2 = '  Grid code min: {}\n  Grid code max: {}'
msg_str3 = '  Data shape: {}'
msg_str4 = '  X shape: {}\n  Y shape: {}'

def get_data_stats(fpath, lt_val):
    msg_str = msg_str0.format(fpath)
    print(msg_str)
    data = pd.read_csv(fpath, header=0)
    msg_str = msg_str1.format(data.Latitude.min(), gravdata.Latitude.max())
    print(msg_str)
    # q1. why add/subtract these values from lat and lon?
    # q2. why only take data less than a certain value?
    data = data[np.abs(data.Latitude + 24.85) < lt_val]
    data = data[np.abs(data.Longitude - 116.1) < lt_val]
    msg_str = msg_str2.format(data.Latitude.min(), gravdata.Latitude.max())
    print(msg_str)
    msg_str = msg_str3.format(data.shape)
    print(msg_str)
    return(data)

def run_gp(data):
    X = np.array([data.Latitude, data.Longitude]).T
    Y = np.array([data.grid_code]).T
    kernel = GPy.kern.Matern32(2)
    msg_str = msg_str4.format(X.shape, Y.shape)
    model = GPy.models.GPRegression(X, Y, kernel)
    model.optimize(messages=True)
    fig = plt.figure(figsize = (10, 10))
    f = model.plot()
    print(model)
    return(X, Y)

# Grav
This seems pretty weird -- the gravity data seems to have a very long length scale and no obvious noise.  But we can see from the contours that there is some structure.  Not sure what to make of that.

In [ ]:
fpath = os.path.join(dir_data, fname_grav)
data = get_data_stats(fpath, lt_val_grav)
X, Y = run_gp(data)

# Mag
Magnetism, on the other hand, has at least some non-zero Gaussian noise to it.  But surely the length scale is kind of out of whack?  And are those repeated points there?

In [ ]:
fpath = os.path.join(dir_data, fname_mag)
data = get_data_stats(fpath, lt_val_mag)
X, Y = run_gp(data)

Oooh looks like they are.  Well, in a way that's useful, if those are real -- in principle they give us the noise scale.  But if it isn't real, it's not clear this would have worked.

In [ ]:
# looking at mag X, Y
dX0 = X[:,0].reshape(36,36)[:,0]
print(dX0)
print(dX0[1:] - dX0[:-1])
dX1 = X[:,1].reshape(36,36)[1,:]
print(dX1)
print(dX1[1:] - dX1[:-1])